### Imports and Settings

In [ ]:
import pickle
import os
import numpy as np
import matplotlib.pyplot as plt

import SAGA
from SAGA.database import FitsTable
from easyquery import Query
from easyquery import QueryMaker
from SAGA import ObjectCuts as C
from SAGA.utils.distance import d2m

from scipy.stats import spearmanr

### Load SAGA Data

In [ ]:
saga = SAGA.QuickStart()
saga.set_default_base_version("paper2")

In [ ]:
hosts = saga.host_catalog.load(include_stats=True, query="paper2_complete")["HOSTID", "K_ABS", "sats_Mr_limit"]

In [ ]:
def annotate_base(base):
    base["Mr_"] = base["r_mag"] - d2m(base["HOST_DIST"])
    base["Mr"] = np.where(np.isfinite(base["Mr"]) & C.has_spec.mask(base), base["Mr"], base["Mr_"])
    base["log_sm"] = np.where(np.isfinite(base["log_sm"]), base["log_sm"], 1.254 + 1.0976 * base["gr"] - 0.4 * base["Mr"])
    return base

In [ ]:
base = SAGA.database.FitsTable("/home/yymao/Documents/Research/SAGA/PaperII/data-archive/saga_base_all.fits").read()
base = saga.host_catalog.construct_host_query("paper2_complete").filter(base)
assert len(np.unique(base["HOSTID"])) == 36

base = annotate_base(base)

In [ ]:
sats = C.is_sat.filter(base, ["HOSTID", "Mr"])
possible_sats = Query(C.basic_cut2, ~C.has_spec, "p_sat_corrected > 0", C.r_abs_limit).filter(base, ["HOSTID", "p_sat_corrected"])

In [ ]:
hosts["sats_correction"] = [QueryMaker.equals("HOSTID", h).filter(possible_sats, "p_sat_corrected").sum() for h in hosts["HOSTID"]] 
hosts["sats_brightest"] = [s.min() if len(s := QueryMaker.equals("HOSTID", h).filter(sats, "Mr")) else 0 for h in hosts["HOSTID"]]
hosts["sats_check"] = [Query(C.r_abs_limit, QueryMaker.equals("HOSTID", h)).count(sats) for h in hosts["HOSTID"]]

hosts["K_PLOT"] = np.where(QueryMaker.equals("HOSTID", "nsa147606").mask(hosts), hosts["K_ABS"] + 0.04, hosts["K_ABS"])

In [ ]:
assert (hosts["sats_Mr_limit"] == hosts["sats_check"]).all()

hosts

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(10,4))


ax[0].axvline(-23.0, ls='--', color='k', alpha=0.5)
ax[0].axvline(-24.6, ls='--', color='k', alpha=0.5)

hosts_this = saga.host_catalog.construct_host_query("paper1_complete").filter(hosts)

ax[0].errorbar(hosts_this["K_PLOT"], hosts_this["sats_Mr_limit"], 
               yerr=(np.zeros(len(hosts_this)), hosts_this["sats_correction"]),
               capsize=0, marker='o', ls='', ms=7, color="limegreen", label="Paper I systems")

hosts_this = Query("sats_Mr_limit > 0").filter(hosts_this)

ax[1].errorbar(hosts_this["sats_brightest"], hosts_this["sats_Mr_limit"], 
               yerr=(np.zeros(len(hosts_this)), hosts_this["sats_correction"]),
               capsize=0, marker='o', ls='', ms=7, color="limegreen", label="Paper I systems")


hosts_this = (~saga.host_catalog.construct_host_query("paper1_complete")).filter(hosts)

ax[0].errorbar(hosts_this["K_PLOT"], hosts_this["sats_Mr_limit"], 
               yerr=(np.zeros(len(hosts_this)), hosts_this["sats_correction"]),
               capsize=0, marker='o', ls='', ms=7, color="g", label="New from this work")

hosts_this = Query("sats_Mr_limit > 0").filter(hosts_this)

ax[1].errorbar(hosts_this["sats_brightest"], hosts_this["sats_Mr_limit"], 
               yerr=(np.zeros(len(hosts_this)), hosts_this["sats_correction"]),
               capsize=0, marker='o', ls='', ms=7, color="g", label="New from this work")

#MW
ax[0].plot(-24, 5, color='k', marker='*', ms=11, zorder=999, ls="")
ax[0].text(-24, 5, 'MW', fontweight='bold', va="bottom", ha="right")

#M31
ax[0].plot(-24.7, 9, color='grey', marker='*', ms=11, zorder=999, ls="")
ax[0].text(-24.65, 9.25, 'M31', color='grey', fontweight='bold', va="bottom", ha="right")

#MW
ax[1].plot(-18.5, 5, color='k', marker='*', ms=11, zorder=999, ls="")

#M31
ax[1].plot(-19.2, 9, color='grey', marker='*', ms=11, zorder=999, ls="")

ax[1].legend(loc="upper right", markerfirst=False, frameon=True, handletextpad=0.1, handlelength=1)

ax[0].set_ylim(-0.25, 11)
ax[1].set_ylim(-0.25, 11)
ax[0].set_ylabel(r'$N_\mathrm{sats} \, (M_{r,o} > -12.3)}$')

ax[0].set_xlim(-24.9, -22.8)
ax[0].set_xticks(np.arange(-24.8, -22.7, 0.4));
ax[0].set_xlabel('Host galaxy  $M_K$ [mag]');

ax[1].set_xlim(-20.5, -13.5)
ax[1].set_xticks(np.arange(-20, -13.9, 1));
ax[1].set_xlabel('Brightest satellite  $M_{r,o}$ [mag]');

plt.tight_layout()
plt.savefig('/home/yymao/Downloads/num_sat_host_mk_new.pdf', bbox_inches="tight")

In [ ]:
spearmanr(hosts["K_ABS"], hosts["sats_Mr_limit"])

In [ ]:
hosts_this = Query("sats_Mr_limit > 0").filter(hosts_this)
spearmanr(hosts_this["sats_brightest"], hosts_this["sats_Mr_limit"])